# Lab03: Frequent itemset mining

- Student ID: 21120123
- Student name: Lê Thanh Thái Quảng

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = set(a) | set(b)
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO:   
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        # Tính support khi đưa itemset vào và đếm trong data, data là 1 dict với key là tid và value là items (items là set)
        support = 0
        for items in self.data.values():
            if set(itemset) <= items:
                support += 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        # search_space là dict với key là item, 
        for itemset in search_space.keys():
            if search_space[itemset]["support"] > self.minSup:
                self.L[k].append(search_space[itemset]["itemset"])
            else:
                search_space[itemset]["pruned"] = True

    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                if k == 1:
                    a_ = [a]
                    b_ = [b]
                else:
                    a_ = a
                    b_ = b
                newset = joinset(a_, b_)
                
                # Second add newset to search_space
                sub_tree[tuple(newset)] = {}
                sub_search_space[tuple(newset)] = {
                    "itemset": newset,
                    "pruned": False,
                    "support": self.computeItemsetSupport(newset)
                }
                
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('data/chess.txt')

In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [{48, 52}, {48, 58}, {56, 29}, {56, 52}, {56, 58}, {66, 60}, {66, 29}, {66, 52}, {66, 58}, {40, 34}, {34, 29}, {34, 52}, {34, 58}, {60, 62}, {40, 62}, {29, 62}, {52, 62}, {58, 62}, {60, 7}, {40, 7}, {29, 7}, {52, 7}, {58, 7}, {36, 60}, {40, 36}, {36, 29}, {36, 52}, {58, 36}, {40, 60}, {60, 29}, {60, 52}, {58, 60}, {40, 29}, {40, 52}, {40, 58}, {52, 29}, {58, 29}, {58, 52}], 3: [{48, 58, 52}, {56, 52, 29}, {56, 58, 29}, {56, 58, 52}, {66, 60, 29}, {66, 60, 52}, {66, 60, 58}, {66, 52, 29}, {66, 58, 29}, {66, 52, 58}, {40, 34, 29}, {40, 34, 52}, {40, 34, 58}, {34, 52, 29}, {34, 58, 29}, {34, 52, 58}, {60, 29, 62}, {60, 62, 52}, {58, 60, 62}, {40, 29, 62}, {40, 52, 62}, {40, 58, 62}, {52, 29, 62}, {58, 29, 62}, {58, 52, 62}, {40, 60, 7}, {60, 29, 7}, {60, 52, 7}, {58, 60, 7}, {40, 29, 7}, {40, 52, 7}, {40, 58, 7}, {52, 29, 7}, {58, 29, 7}, {58, 52, 7}, {40, 36, 60}, {36, 29, 60}, {36, 60, 52}, {58, 36, 60}, {40

### Answer questions here:
**Why don't we compute support of items while reading data?**
- Trong quá trình đọc dữ liệu, đối với lớn có thể tạo rất rất nhiều itemset với 1, 2, 3, ... phần tử, nếu như tính support của toàn bộ thì rất tốn kém bởi không phải toàn bộ itemset đều là phổ biến. Thay vào đó trong quá trình đọc, sau khi đọc 1 dòng transaction, chỉ cần tính support của item trên dòng đó.

**why should we do sort**
- Sắp xếp các itemset trong ```self.s``` theo thứ tự tăng dần của support. Việc sắp xếp các itemset theo thứ tự tăng dần của support giúp trong việc tối ưu hóa ```prune``` vì có thể giảm thiểu số lượng phép so sánh cần thực hiện và tăng hiệu suất của thuật toán.

**study about python set and its advantages ?**
- Python set là tập hợp các phần tử không trùng nhau và không có thứ tự (không thể truy cập phần tử bằng chỉ số). Set cung cấp các phép toán tập hợp như: giao, hợp, hiệu, ...
- Lợi ích của set: Loại bỏ các phần tử trùng lặp tự động, giúp ích khi join 2 set. Không chứa phần tử trùng nhau nên khi đếm support không phải xử lí trường hợp trùng nhau. Xác định dễ dàng 1 set có xuất hiện trong 1 set khác hay không.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- Chỉ cần đánh dấu pruned="True" hay pruned="False" các itemset không phổ biến mà không cần phải xóa khỏi search_space và tree; khi duyệt cây chỉ cần bỏ qua nhánh có đánh dấu pruned="True". Việc xóa các itemset này có thể có thể có sai sót trong qua trình thực hiện thuật toán, thuật toán này là thuật toán đệ qui nên trong quá trình có thể sẽ có sử dụng đến các nhanh bị đánh dấu pruned="True" (Cụ thể tại câu lệnh " if search_space[a]['pruned'])

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**
- Apriori tìm các tập phổ biến bằng cách: tạo tập ứng viên có $k$ phần tử từ các tập phổ biến có $k-1$ sau đó tính support của các tập ứng viên rồi chọn ra các tập phổ biến. Như vậy với Apriori số lượng tập ứng viên tạo ra rất nhiều.
- Tree Projection sử dụng cấu trúc cây để tìm tập phổ biến. Đầu tiên xây dựng một cây mà mỗi nút của cây tương ứng với một item trong tập dữ liệu ban đầu. Tại một nút ở cây có itemset có độ dài $k$, nếu support của itemset đó không thỏa minsup thì nút đó sẽ bị pruned="True", nếu thỏa minsup thì join itemset tại nút đó với itemset của lần lượt nút có cùng độ sâu để tạo thành itemset mới có độ dài $k+1$. Tiếp tục đến nút mới tạo ra và lặp lại tương tự (đệ qui). Như vậy thuật toán sẽ không tạo ra toàn bộ các itemset ứng viên và thuật toán chỉ duyệt qua dữ liệu một lần duy nhất.


# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [6]:
def readData2(path):
    data={}
    s=defaultdict(lambda: 0)
    with open(path,'rt') as f:
        attrs = f.readline().split(",")
        attrs[-1] = attrs[-1].rstrip("?\n")
        tid=1;
        for line in f:
            values = line.split(",")
            values[-1] = values[-1].rstrip(".\n")

            # itemset là 1 string có được bằng cách ghép tên thuộc tính và giá trị
            # lý do: Có thuộc tính có thể có cùng miền giá trị
            # ví dụ: Int'l Plan, VMail Plan đều có miền giá trị là {yes, no}
            # Nếu như cả Int'l Plan, VMail Plan đều có giá trị "no" khi tạo set sẽ bị mất 1 giá trị "no",
            # mặc dù ý nghĩa Int'l Plan, VMail Plan là khác nhau
            itemset = [f"{attrs[i]}={values[i]}" for i in range(len(attrs))]
            itemset = set(itemset)
            
            for item in itemset:  
                s[item]+=1     
            data[tid]= itemset
            tid+=1
    
    return data, s

In [7]:
# minSup=1000
data2, s2 = readData2("data/churn.txt")
a2 = TP(data=data2, s=s2, minSup=1000)
print(a2.miningResults())

{1: [['CustServ Calls=1'], ['Area Code=415'], ['VMail Message=0'], ['VMail Plan=no'], ['Churn=False'], ["Int'l Plan=no"]], 2: [{'Churn=False', 'CustServ Calls=1'}, {'CustServ Calls=1', "Int'l Plan=no"}, {'Area Code=415', 'VMail Message=0'}, {'Area Code=415', 'VMail Plan=no'}, {'Churn=False', 'Area Code=415'}, {'Area Code=415', "Int'l Plan=no"}, {'VMail Message=0', 'VMail Plan=no'}, {'Churn=False', 'VMail Message=0'}, {'VMail Message=0', "Int'l Plan=no"}, {'Churn=False', 'VMail Plan=no'}, {'VMail Plan=no', "Int'l Plan=no"}, {'Churn=False', "Int'l Plan=no"}], 3: [{'Area Code=415', 'VMail Message=0', 'VMail Plan=no'}, {'Area Code=415', 'VMail Message=0', "Int'l Plan=no"}, {'Area Code=415', 'VMail Plan=no', "Int'l Plan=no"}, {'Churn=False', 'Area Code=415', "Int'l Plan=no"}, {'Churn=False', 'VMail Message=0', 'VMail Plan=no'}, {'VMail Message=0', 'VMail Plan=no', "Int'l Plan=no"}, {'Churn=False', 'VMail Message=0', "Int'l Plan=no"}, {'Churn=False', 'VMail Plan=no', "Int'l Plan=no"}], 4: [{

# 4 References

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
